In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.isnull().sum() #no missing data

In [ ]:
#statistical analysis
df.describe().transpose()

Exploratory data analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df["price"])


0-1 million $ most of the houses

In [ ]:
sns.countplot(x=df["bedrooms"])

there are bedrooms with 10,11,33 etc

In [ ]:
#checking what are highly correlated features
df.corr()["price"].sort_values()

sqft_living has high corr

In [ ]:
sns.scatterplot(x="price",y="sqft_living",data=df)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x="bedrooms",y="price",data=df)

according to our countplot 2-5 bedrooms are lot. can see a huge variety of price in 3-6 bedrooms

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x="price",y="long",data=df)

long = -122.2 tend to be a high expensive area

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x="price",y="lat",data=df)

47.6 - 47.7 tend to have an expensive area

In [ ]:
#most expensive houses
df.sort_values("price",ascending=False).head(20)

In [ ]:
sns.boxplot(x="waterfront",y="price",data=df)

Feature Engineering

In [ ]:
df = df.drop("id",axis=1)

In [ ]:
#dates are string, we convert to date time object
df["date"]= pd.to_datetime(df["date"])

In [ ]:
#exracting yr and month
df["year"] = df['date'].apply(lambda date: date.year)
df["month"] = df['date'].apply(lambda date: date.month)

In [ ]:
df.head()

In [ ]:
#checking the corr of the month and price
plt.figure(figsize=(10,6))
sns.boxplot(x='month', y="price", data=df)

In [ ]:
df.groupby("month").mean()["price"].plot()

In [ ]:
df = df.drop("date",axis=1)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.value_counts("zipcode")

70 categories are too much, we drop zipcode

In [ ]:
df = df.drop("zipcode",axis=1)

In [ ]:
df["yr_renovated"].value_counts()

In [ ]:
df["sqft_basement"].value_counts()

Data preprocessing and creating a model

In [ ]:
X= df.drop("price",axis=1).values
y=df["price"].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
scaler=MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test=scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X_train.shape

good to have 19 nodes in our input layer


In [ ]:
model=Sequential()

model.add(Dense(19,activation="relu"))
model.add(Dense(19,activation="relu"))
model.add(Dense(19,activation="relu"))
model.add(Dense(19,activation="relu"))
model.add(Dense(1))

In [ ]:
model.compile(optimizer="adam",loss="mse")

In [ ]:
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),batch_size=128, epochs=400)

In [ ]:
loss=pd.DataFrame(model.history.history)
loss.plot()

we could continue training for more time, no indication of overfitting

In [ ]:
pred = model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

In [ ]:
mean_absolute_error(y_test,pred)

In [ ]:
df["price"].describe()

In [ ]:
5.400881e+05

we are off by around 20%

In [ ]:
explained_variance_score(y_test,pred)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(y_test,pred)

0-2 million have a good fitting